<a href="https://colab.research.google.com/github/thehimalayanleo/Private-Machine-Learning/blob/master/DifferentialPrivacy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 

In [0]:
num_individuals = 5000
database = torch.rand(num_individuals) > 0.5
database

tensor([1, 1, 1,  ..., 0, 1, 1], dtype=torch.uint8)

In [0]:
def gen_parallel_dbs(db):
  pdbs = []
  len_db = len(db)
  for indx in range(len_db):
    temp_db = torch.cat((db[:indx], db[indx+1:]))
    pdbs.append(temp_db)
  return pdbs
    
parallel_database = gen_parallel_dbs(database)
print(len(parallel_database[-1]))

4999


In [0]:
def gen_db_pdbs(n_indi):
  db = torch.rand(n_indi) > 0.5
  return db, gen_parallel_dbs(db)

In [0]:
## Let us see how the output of the same query changes with different parallel databases
## Thus, we wish to find out the sensitivity of the database, which we do by finiding out
## the maximum difference in queries over the parallel databases.

# Simplest query possible, a sum!
def query_sum(db):
  return db.sum()

In [0]:
db, pdbs = gen_db_pdbs(30)
print(query_sum(db))

tensor(21)


In [0]:
# Let us find the sensitivity
query_outputs = []
for pdb in pdbs:
  #print(query_sum(pdb))
  query_outputs.append(query_sum(pdb))
  
torch.sort(torch.tensor(query_outputs))
#print(query_outputs[0])
sensitivity = query_outputs[len(query_outputs)-1]-query_outputs[0]
print(sensitivity)

tensor(1)


In [0]:
def sensitivity_calc(query, n_indi):
  db, pdbs = gen_db_pdbs(n_indi)
  sensitivity = 0
  original_query = query(db)
  #print(original_query)
  for pdb in pdbs:
    query_out = query(pdb)
    diff = torch.abs(query_out - original_query)
    #print(diff)
    if diff > sensitivity:
      sensitivity = diff
  #print(sensitivity)
  return db, pdbs, sensitivity

In [0]:
db, pdbs, sensitivity = sensitivity_calc(query_sum, 20)
print(sensitivity)

tensor(11)
tensor(1)


In [0]:
def query_mean(db):
  return db.float().mean()

In [0]:
db, pdbs, sensitivity = sensitivity_calc(query_mean, 10)
print(sensitivity)

tensor(0.6000)
tensor(0.0667)


In [0]:
def query_threshold(db, threshold = 5):
  #print(threshold)
  return (db.sum() > threshold).float()

In [0]:
#db, pdbs = gen_db_pdbs(10)
#print(pdbs)
#for pdb in pdbs:
  #print(sen(pdb))
  #print()
  
_, pdbs, s= sensitivity_calc(query_threshold, 11)
#print(pdbs)
print(s)

0


In [0]:
## Differencing Attack. Find sum of a single individual. Take sum of entire db and take sum without
## that individual

db, _ = gen_db_pdbs(100)
def differencing_attack (db, indx, query, thresh=5):
  if query == query_threshold:
    return torch.abs(query(db, thresh)-query(torch.cat((db[:indx], db[indx+1:])), thresh))
  return torch.abs(query(db)-query(torch.cat((db[:indx], db[indx+1:]))))

print(differencing_attack(db, 10, query_sum))
print(differencing_attack(db, 10, query_mean))
#query_threshold.threshold = 49
print(differencing_attack(db, 10, query_threshold, 50))


tensor(0)
tensor(0.0059)
tensor(0.)
